In [11]:
import json
from pathlib import Path

p_cap_json = Path('/data/gunsbrother/prjs/ltvu/llms/Video-LLaVA/ltvu/captions/test/step1/0b20e242-a496-4662-a3e7-645bcecdbe55/cbdc37c7-820a-5bb3-a597-53ca31a13a6f.json')
with p_cap_json.open() as f:
    cap_json = json.load(f)
query = cap_json['query']
prompts = cap_json['prompts']
for caps in cap_json['captions']:
    s, e = caps['interval']['start_sec'], caps['interval']['end_sec']
    print(f'{s}s ~ {e}s')
    for prompt, cap in zip(prompts, caps['captions']):
        prompt = prompt.format(query=query)
        print(f'{prompt:40s}  {cap}')
    print()

0.0s ~ 3.0s
where did I pour water?                   I poured water into a pot on the stove.</s>
Did that appear in the video?             Yes, the video shows a person pouring water into a pot on the stove.</s>

3.0s ~ 6.0s
where did I pour water?                   I poured water into a pot on the stove.</s>
Did that appear in the video?             Yes, the video shows a person pouring water into a pot on the stove.</s>

6.0s ~ 9.0s
where did I pour water?                   I poured water in the sink.</s>
Did that appear in the video?             Yes, the video shows a person pouring water into the sink.</s>

9.0s ~ 12.0s
where did I pour water?                   I poured water into a glass and then poured it into a pot.</s>
Did that appear in the video?             Yes, the video shows a person pouring water into a glass and then pouring it into a pot.</s>

12.0s ~ 15.0s
where did I pour water?                   I poured water into the sink.</s>
Did that appear in the video?       

In [78]:
import re
import json
from pathlib import Path
import numpy as np
%cd /data/gunsbrother/prjs/ltvu/llms/Video-LLaVA/ltvu
from intervaltree import IntervalTree

p_caps_dir = Path('/data/gunsbrother/prjs/ltvu/llms/Video-LLaVA/ltvu/captions/test/step1')
# p_cap_json = Path('/data/gunsbrother/prjs/ltvu/llms/Video-LLaVA/ltvu/captions/test/step1/0b20e242-a496-4662-a3e7-645bcecdbe55/cbdc37c7-820a-5bb3-a597-53ca31a13a6f.json')

def get_caption_scores(p_cap_json):
    with p_cap_json.open() as f:
        cap_json = json.load(f)

    t = IntervalTree()
    for caps in cap_json['captions']:
        s, e = caps['interval']['start_sec'], caps['interval']['end_sec']
        last_caption = caps['captions'][-1]
        has_yes = bool(re.findall(r'^yes(?=,)', last_caption, flags=re.IGNORECASE))
        has_no = bool(re.findall(r'^no(?=,)', last_caption, flags=re.IGNORECASE))
        assert has_yes ^ has_no, f'{s:5.1f}s ~ {e:5.1f}s {last_caption}'
        t[s:e] = has_yes
    s, e = cap_json['gt']['start_sec'], cap_json['gt']['end_sec']
    t_gt = t[s:e]
    TP = sum([iv.data for iv in t_gt])
    FP = sum([iv.data for iv in t - t_gt])
    FN = sum([not iv.data for iv in t_gt])
    recall = round(TP / (TP + FN + 1e-7), 6)
    precision = round(TP / (TP + FP + 1e-7), 6)
    query = cap_json['query']
    print(f'{p_cap_json.stem} | {TP:3d}/{FP:3d}/{FN:3d} | {recall:6.1%} {precision:6.1%} | {query}')
    return recall, precision

recalls, precisions = [], []
for p_cap_json in p_caps_dir.glob('**/*.json'):
    recall, precision = get_caption_scores(p_cap_json)
    recalls.append(recall)
    precisions.append(precision)
np.mean(recalls), np.mean(precisions)

/data/gunsbrother/prjs/ltvu/llms/Video-LLaVA/ltvu
1b05a71e-91fd-5669-af48-40b31602521d |   1/222/  0 | 100.0%   0.4% | where was the sandals?
02c81446-fe50-5adb-b202-c0f08013843a |   0/ 21/  1 |   0.0%   0.0% | who did I talk to in the field?
25e1c27b-f9fd-5fa7-a914-5a849260ba04 |   1/313/  0 | 100.0%   0.3% | where was the cloth?
fafa2264-646d-5d2b-8e77-b283a0beb13a |   1/210/  0 | 100.0%   0.5% | what color was the bag?
86a7510d-1725-5461-adf2-c9a35db90452 |   0/  6/  1 |   0.0%   0.0% | where was the phone before I picked it?
35720a20-664b-52ae-8931-364950b219ec |   2/146/  0 | 100.0%   1.4% | where was the pink flower?
af19b35c-eca8-56ca-a6b3-8690883ee703 |   0/  9/  1 |   0.0%   0.0% | where did I put the phone?
851b598f-b371-53fd-8d16-230f1e7be3fc |   3/147/  0 | 100.0%   2.0% | in what table did I see the remote?
2de695b5-1ed8-5856-82df-bfb8d05759e9 |   1/ 21/  1 |  50.0%   4.5% | what appliance did I press?
2bfe6d47-cb55-512f-9a37-f7caa8ad9fc6 |   2/ 75/  0 | 100.0%   2.6% | wh

(0.38247422680412363, 0.024211927835051546)